### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

print("--- 콘텐츠분쟁해결 RAG 시스템 구축 시작 ---")

# 0단계: PDF 문서 로드
pdf_path = "./data/콘텐츠분쟁해결_사례.pdf"
try:
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    print(f"'{pdf_path}'에서 {len(docs)} 페이지 문서를 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{pdf_path}' 파일을 찾을 수 없습니다. 경로를 확인해 주세요.")
    print("시스템을 종료합니다.")
    exit() # 파일이 없으면 스크립트 종료

# 1단계: 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        # 법률 사례는 1200-1800자 권장
    chunk_overlap=300,      # 사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",    # 법률 문서 섹션 구분자
        "\n【쟁점사항】",    # 쟁점 부분 구분
        "\n【처리경위】",    # 처리 과정 구분
        "\n【처리결과】",    # 결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
splits = text_splitter.split_documents(docs)
print(f"문서가 {len(splits)}개의 청크로 분할되었습니다.")


In [ ]:
# 2단계: 임베딩 모델 설정 (OpenAI API 사용)
# OpenAIEmbeddings는 OPENAI_API_KEY 환경 변수를 자동으로 사용합니다.
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large", # 한국어 법률 용어에 최적
    dimensions=1536                 # 고성능 임베딩 차원
)
print("OpenAI 임베딩 모델 'text-embedding-3-large'를 설정했습니다.")

# 3단계: 벡터 저장소 및 검색기 설정 (이전과 동일)
print("벡터 저장소(FAISS)를 생성하고 문서 임베딩을 시작합니다...")
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
print("벡터 저장소 생성 완료.")

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)
print("검색기(Retriever)를 설정했습니다. 상위 5개 유사 문서를 검색합니다.")

# 4단계: LLM 설정 (OpenAI API 사용)
# base_url을 제거하여 OpenAI 기본 API 엔드포인트를 사용합니다.
llm = ChatOpenAI(
    model="gpt-4o",         # 또는 "gpt-4o-mini" (경제적), "gpt-3.5-turbo" 등
    temperature=0.2,        # 법률 조언은 정확성 중시
    max_tokens=2000         # 충분한 답변 길이
)
print(f"LLM 모델 '{llm.model_name}' (OpenAI API)을 설정했습니다. Temperature: {llm.temperature}")


In [ ]:

# 5단계: 법률 자문 프롬프트 작성
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다.
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print("법률 자문 프롬프트를 작성했습니다.")

# 6단계: QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 검색된 문서들을 하나의 큰 컨텍스트로 합쳐서 LLM에 전달
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True # 참조 문서도 함께 반환하여 답변의 근거를 확인 가능
)
print("QA 체인을 생성했습니다.")

# 7단계: 테스트 질문 작성
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]
print("테스트 질문 목록을 준비했습니다.")

# 8단계: 분쟁 유형 분류 함수 (선택)
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

print("분쟁 유형 분류 함수를 정의했습니다.")


In [ ]:

# 최종 실행: 각 테스트 질문에 대해 법률 자문 제공
print("\n--- 법률 자문 시작 ---")
for i, question in enumerate(test_questions):
    print(f"\n--- 질문 {i+1}: {question} ---")
    dispute_type = classify_dispute_type(question)
    print(f"분쟁 유형 (분류): {dispute_type}")

    result = qa_chain.invoke({"query": question}) # RetrievalQA 체인은 'query' 키를 사용합니다.

    print("\n[전문 법률 조언]")
    print(result["result"])

    print("\n[참조된 문서 출처]")
    if result["source_documents"]:
        for j, doc in enumerate(result["source_documents"]):
            print(f"  - 문서 {j+1}: 페이지 {doc.metadata.get('page')}, 소스: {doc.metadata.get('source')}")
    else:
        print("  참조된 문서가 없습니다.")

print("\n--- 모든 법률 자문이 완료되었습니다 ---")